# LDA Model Building 

## Topic modelling using tweets of all users


In [1]:
import pandas as pd
import numpy as np
import re

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore

import spacy
import en_core_web_lg

from tqdm import tqdm_notebook as tqdm


# Visualisations
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt 
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls

## Loading Data

In [19]:
tweet_df = pd.read_csv('Data/tweet_climate_change_processed.csv')
tweet_df

,ID,Name,Tweet,Processed_text,Processed_token,Bigram
0,18257804,stateless,"we’re pretty lucky, all things considered, whe...",pretty lucky thing consider compare place clim...,"['pretty', 'lucky', 'thing', 'consider', 'comp...","['pretty', 'lucky', 'thing', 'consider', 'comp..."
1,1246325069841723392,TsaiJilly,#UN75 survey found that respondents in all reg...,un survey find respondent region identify cl...,"['un', 'survey', 'find', 'respondent', 'region...","['find', 'respondent', 'region', 'identify', '..."
2,1248988647812222978,Beatric54184322,"All hat, no policy #climatechange #insiders ht...",hat policy climatechange insider,"['hat', 'policy', 'climatechange', 'insider']","['hat', 'policy', 'climatechange', 'insider']"
3,240072798,LugubriousLarry,Two great stories on #Maine this weekend: firs...,great story maine weekend important piece jan...,"['great', 'story', 'maine', 'weekend', 'import...","['great', 'story', 'maine', 'weekend', 'import..."
4,1124447266205503488,All435Reps,The evidence is right in front of us. Temperat...,evidence right temperature get hot climatechan...,"['evidence', 'right', 'temperature', 'get', 'h...","['evidence', 'right', 'temperature', 'get', 'h..."
...,...,...,...,...,...,...
67401,1329260903527718913,Jillian18277886,"Once we get through #COVID, let's not forget w...",covid let forget go die climatechange,"['covid', 'let', 'forget', 'go', 'die', 'clima...","['let', 'forget', 'go', 'die', 'climatechange']"
67402,19435213,wildweatherdan,"At the end of the last Ice Age, people changed...",end ice age people change clothe literally cli...,"['end', 'ice', 'age', 'people', 'change', 'clo...","['end', 'ice', 'age', 'people', 'change', 'clo..."
67403,1401488848635502592,AnonWatchers,"Coral #reefs ,rainforest of the ocean\n🐠\n#cli...",coral reef rainforest ocean climatechange co...,"['coral', 'reef', 'rainforest', 'ocean', 'clim...","['coral_reef', 'rainforest', 'climatechange', ..."
67404,18027211,dennissweatt,"What is #climatechange? \n\nWets are wetter, d...",climatechange wet wetter dry dryer hot hot col...,"['climatechange', 'wet', 'wetter', 'dry', 'dry...","['climatechange', 'wet', 'wetter', 'dry', 'dry..."


## Data Transformation : Corpus and Dictionary 

In [135]:
# Convert stringfied list back to list
tweet_df['Processed_token'] = tweet_df['Processed_token'].apply(eval)

In [137]:
# Create a mapping of word IDs to words
id2word = corpora.Dictionary(tweet_df['Processed_token'])
print(len(id2word))

36785


In [138]:
# Filter extremes
id2word.filter_extremes(no_below = 3, no_above = .99)
print(len(id2word))

20608


In [139]:
# Turns each tweet into a bag of words
corpus = [id2word.doc2bow(tweet) for tweet in tweet_df['Processed_token']]

### Bigram 

In [20]:
# Convert stringfied list back to list
tweet_df['Bigram'] = tweet_df['Bigram'].apply(eval)

In [21]:
# Create a mapping of word IDs to words
id2word_bi = corpora.Dictionary(tweet_df['Bigram'])
print(len(id2word_bi))

28650


In [22]:
# Filter extremes
id2word_bi.filter_extremes(no_below = 3, no_above = .99)
print(len(id2word_bi))

16988


In [23]:
# Turns each tweet into a bag of words
corpus_bi = [id2word_bi.doc2bow(tweet) for tweet in tweet_df['Bigram']]

## Base Model 

- Dirichlet hyperparameter alpha: Document-Topic Density = 1.0(default)

- Dirichlet hyperparameter beta: Word-Topic Density = 1.0(default)


In [12]:
# LDA model
base_lda = LdaMulticore(corpus = corpus, num_topics = 10, 
                       id2word = id2word, random_state = 100,
                       chunksize = 100, passes = 10, 
                        per_word_topics = True)

In [13]:
# Filter for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_lda.print_topics()]

In [14]:
# Create topics
topics = [' '.join(t[0:10]) for t in words]

In [15]:
# Get the topics
for id, t in enumerate(topics):
    print(f"------Topic {id} ------")
    print(t, end = "\n\n")

------Topic 0 ------
sustainability environment energy nature climateaction renewableenergy climateemergency renewable actonclimate climatecrisis

------Topic 1 ------
climate climatecrisis change climateaction people c climateemergency global world cause

------Topic 2 ------
climate heat climatecrisis heatwave change record year extreme weather temperature

------Topic 3 ------
need fuel study fossil human climate eu carbon burn g

------Topic 4 ------
cop renewable june state ev energy tesla finance renewableenergy implement

------Topic 5 ------
health space know good action news tackle reach set environment

------Topic 6 ------
normal sea rise level world discuss w decade fight unep

------Topic 7 ------
climate change science impact solution join community esg july new

------Topic 8 ------
carbon forest climate change tree co world emission climatecrisis plant

------Topic 9 ------
water drought wildfire fire crisis ocean climatecrisis lead heat process



### Model Visualisation

In [17]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
gensimvis.prepare(base_lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.074832 -0.002623       1        1  14.369594
2     -0.056875 -0.133230       2        1  12.524213
7     -0.162463 -0.013741       3        1  11.395481
1     -0.067413 -0.098497       4        1  11.204116
5     -0.159734 -0.028375       5        1   9.137863
4      0.156333  0.340656       6        1   9.063032
3     -0.144201  0.131001       7        1   8.950283
9      0.147304 -0.136023       8        1   8.536788
0      0.076659  0.089330       9        1   7.499589
6      0.285224 -0.148497      10        1   7.319042, topic_info=              Term          Freq         Total Category  logprob  loglift
90         climate  16844.000000  16844.000000  Default  30.0000  30.0000
107          water   3098.000000   3098.000000  Default  29.0000  29.0000
352         normal   2358.000000   2358.000000  Default  28.0000  28.0000
54   climatecrisis  10599.000000  10599.000000  Default  27.0000  27.0000
301           heat   3397.000000   3397.000000  Default  26.0000  26.0000
..             ...           ...           ...      ...      ...      ...
349           high    731.145605   1775.463973  Topic10  -4.7083   1.7275
508         effect    744.943972   1997.249563  Topic10  -4.6896   1.6285
194           tree    768.390033   2490.745185  Topic10  -4.6586   1.4387
157     california    584.343290   1188.376845  Topic10  -4.9324   1.9048
365           fire    601.627541   2661.422147  Topic10  -4.9033   1.1277

[595 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
3486      1  0.997528    account
52        1  0.035782     action
52        2  0.050094     action
52        3  0.214333     action
52        4  0.139549     action
...     ...       ...        ...
88        9  0.139413       year
2028      7  0.996938  yesterday
2033      2  0.286391      youth
2033      7  0.712149      youth
3207      4  0.997387  zerowaste

[1048 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 3, 8, 2, 6, 5, 4, 10, 1, 7])

### Model Evaluation

In [140]:
# Compute Perplexity
# a measure of how good the model is. lower the better
base_perplexity = base_lda.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=base_lda, texts= tweet_df['Processed_token'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -8.534126351796964

Coherence Score:  0.329901875555909


### Bigram 

In [25]:
# LDA model
base_lda_bi = LdaMulticore(corpus = corpus_bi, num_topics = 10, 
                       id2word = id2word_bi, random_state = 100,
                       chunksize = 100, passes = 10, 
                        per_word_topics = True)

#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
gensimvis.prepare(base_lda_bi, corpus_bi, id2word_bi)


# Compute Perplexity
# a measure of how good the model is. lower the better
base_perplexity_bi = base_lda_bi.log_perplexity(corpus_bi)
print('\nPerplexity: ', base_perplexity_bi) 

# Compute Coherence Score
coherence_model_bi = CoherenceModel(model=base_lda_bi, texts= tweet_df['Bigram'], 
                                   dictionary=id2word_bi, coherence='c_v')
coherence_lda_model_base_bi = coherence_model_bi.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base_bi)


Perplexity:  -8.085208900341186

Coherence Score:  0.33407135777190855


## Hyperparameter Tuning

### Parameters to tune

1. Number of Topics (K)

2. Dirichlet hyperparameter alpha: Document-Topic Density (a)

3. Dirichlet hyperparameter beta: Word-Topic Density (b)

In [141]:
def coherence_score(corpus, id2word, k, a, b):
    
    lda_model = LdaMulticore(corpus = corpus,  
                             id2word = id2word,
                             num_topics = k,
                             alpha = a,
                             eta = b,
                             random_state = 100,
                             chunksize = 100, 
                             passes = 10)
    coherence_model_lda = CoherenceModel(model = lda_model,
                                        texts = tweet_df['Processed_token'],
                                        dictionary = id2word,
                                        coherence = 'c_v')
    
    return coherence_model_lda.get_coherence()

In [142]:
# Topic number k range
min_topics = 3
max_topics = 12
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha 
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

model_results_ = {'Topics' : [],
                'Alpha' : [],
                'Beta' : [],
                'Coherence' : []
                }


if 1 == 1:
    pbar = tqdm(total = 270)
    
    # iterate through number of topics
    for k in topics_range:
        # iterate through alpha values
        for a in alpha:
            # iterate through beta values
            for b in beta:
                c = coherence_score(corpus = corpus, 
                                    id2word = id2word,
                                    k = k, a = a, b = b)
                
                model_results_['Topics'].append(k)
                model_results_['Alpha'].append(a)
                model_results_['Beta'].append(b)
                model_results_['Coherence'].append(c)
                
                pbar.update(1)
                
model_result_ = pd.DataFrame(model_results_)
pbar.close()

/var/folders/zr/3t3krv2s4b3290bm28xqz3rm0000gn/T/ipykernel_15545/1928177338.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  pbar = tqdm(total = 270)


  0%|          | 0/270 [00:00<?, ?it/s]

In [144]:
model_result_.sort_values(by = ['Coherence'], ascending = False)

,Topics,Alpha,Beta,Coherence
177,8,asymmetric,0.61,0.468267
98,6,0.31,0.91,0.445448
86,5,asymmetric,0.31,0.431366
243,11,0.01,0.91,0.428462
113,6,symmetric,0.91,0.423610
...,...,...,...,...
1,3,0.01,0.31,0.233670
0,3,0.01,0.01,0.230794
8,3,0.31,0.91,0.203296
2,3,0.01,0.61,0.200856


In [146]:
model_result_.to_csv('Model_results/LDA_tuning_results.csv', index = False)

### Bigram

In [27]:
def coherence_score(corpus, id2word, k, a, b):
    
    lda_model = LdaMulticore(corpus = corpus,  
                             id2word = id2word,
                             num_topics = k,
                             alpha = a,
                             eta = b,
                             random_state = 100,
                             chunksize = 100, 
                             passes = 10)
    coherence_model_lda = CoherenceModel(model = lda_model,
                                        texts = tweet_df['Bigram'],
                                        dictionary = id2word,
                                        coherence = 'c_v')
    
    return coherence_model_lda.get_coherence()

In [45]:
# Topic number k range
min_topics = 3
max_topics = 12
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha 
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

model_results = {'Topics' : [],
                'Alpha' : [],
                'Beta' : [],
                'Coherence' : []
                }


if 1 == 1:
    pbar = tqdm(total = 270)
    
    # iterate through number of topics
    for k in topics_range:
        # iterate through alpha values
        for a in alpha:
            # iterate through beta values
            for b in beta:
                c = coherence_score(corpus = corpus_bi, 
                                    id2word = id2word_bi,
                                    k = k, a = a, b = b)
                
                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(c)
                
                pbar.update(1)
                
model_result = pd.DataFrame(model_results)
pbar.close()

/var/folders/zr/3t3krv2s4b3290bm28xqz3rm0000gn/T/ipykernel_15545/851444230.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  pbar = tqdm(total = 270)


  0%|          | 0/270 [00:00<?, ?it/s]

In [145]:
model_result = pd.DataFrame(model_results)
model_result.to_csv('Model_results/LDA_tuning_results_bi.csv', index = False)

In [97]:
model_result.sort_values(by = ['Coherence'], ascending = False)

,Topics,Alpha,Beta,Coherence
44,4,0.61,symmetric,0.388452
117,6,asymmetric,0.61,0.382405
131,7,0.61,0.31,0.374976
191,9,0.61,0.31,0.374794
76,5,0.91,0.31,0.371659
...,...,...,...,...
26,3,asymmetric,0.31,0.281213
61,5,0.01,0.31,0.276921
64,5,0.01,symmetric,0.272813
27,3,asymmetric,0.61,0.261185


## Best Model

In [165]:
# LDA model 
best_lda = LdaMulticore(corpus = corpus, 
                           num_topics = 8, 
                           id2word = id2word, 
                           random_state = 100,
                           chunksize = 100,
                           passes = 10, 
                          alpha = 'asymmetric',
                          eta = 0.61)

#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
vis =  gensimvis.prepare(best_lda, corpus, id2word)

/Users/jlee/earth_org/e_venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/jlee/earth_org/e_venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/jlee/earth_org/e_venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/jlee/earth_org/e_venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/jlee/earth_org/e_venv/lib/python3.8/site-packages/past/builti

In [166]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.097899  0.083338       1        1  24.618028
1     -0.149573 -0.015065       2        1  16.293889
2     -0.241449 -0.011808       3        1  15.312915
7      0.120421  0.097052       4        1  13.418128
3     -0.008246 -0.131093       5        1   9.025853
4      0.112717  0.236990       6        1   7.623795
5      0.187460 -0.149366       7        1   6.929771
6      0.076569 -0.110048       8        1   6.777621, topic_info=              Term          Freq         Total Category  logprob  loglift
54   climatecrisis  11272.000000  11272.000000  Default  30.0000  30.0000
301           heat   3827.000000   3827.000000  Default  29.0000  29.0000
368       heatwave   2665.000000   2665.000000  Default  28.0000  28.0000
995         decade   2020.000000   2020.000000  Default  27.0000  27.0000
300        extreme   1895.000000   1895.000000  Default  26.0000  26.0000
..             ...           ...           ...      ...      ...      ...
479        science    289.025932   2414.113723   Topic8  -5.5595   0.5690
182           read    276.287504   2285.662503   Topic8  -5.6046   0.5786
762         health    263.619173   1686.626861   Topic8  -5.6515   0.8356
157     california    255.331900   1278.089500   Topic8  -5.6835   1.0810
89          change    261.875211   7907.539958   Topic8  -5.6582  -0.7162

[531 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
4612      1  0.005701  aaas
4612      2  0.005701  aaas
4612      3  0.005701  aaas
4612      4  0.005701  aaas
4612      5  0.005701  aaas
...     ...       ...   ...
2018      4  0.006566  zoom
2018      5  0.006566  zoom
2018      6  0.006566  zoom
2018      7  0.971811  zoom
2018      8  0.006566  zoom

[3552 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 8, 4, 5, 6, 7])

In [167]:
pyLDAvis.save_html(vis, 'Visualisation/best_model_vis.html')

### Bigram

In [98]:
# LDA model
best_lda_bi = LdaMulticore(corpus = corpus_bi, 
                           num_topics = 4, 
                           id2word = id2word_bi, 
                           random_state = 100,
                           chunksize = 100,
                           passes = 10, 
                          alpha = 0.61,
                          eta = 'symmetric')

#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
gensimvis.prepare(best_lda_bi, corpus_bi, id2word_bi)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.117031 -0.224711       1        1  27.919696
2      0.262873  0.105310       2        1  25.205893
0      0.076123 -0.092182       3        1  24.639779
1     -0.221965  0.211584       4        1  22.234632, topic_info=              Term          Freq         Total Category  logprob  loglift
80         climate  16503.000000  16503.000000  Default  30.0000  30.0000
79          change   8287.000000   8287.000000  Default  29.0000  29.0000
48   climatecrisis  12216.000000  12216.000000  Default  28.0000  28.0000
160           tree   3514.000000   3514.000000  Default  27.0000  27.0000
83     environment   3183.000000   3183.000000  Default  26.0000  26.0000
..             ...           ...           ...      ...      ...      ...
699             co   1085.863992   2290.881286   Topic4  -5.1817   0.7570
500  climateaction   1312.142824   5281.822377   Topic4  -4.9924   0.1109
568   biodiversity    851.780669   1371.349823   Topic4  -5.4245   1.0273
138    globalwarme   1062.666875   3331.931899   Topic4  -5.2033   0.3607
78            year   1020.621768   4471.636371   Topic4  -5.2437   0.0262

[246 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
46        1  0.436022        action
46        3  0.361081        action
46        4  0.202925        action
1984      3  0.996552      activist
96        1  0.073591  actonclimate
...     ...       ...           ...
78        2  0.663068          year
78        3  0.001118          year
78        4  0.228328          year
907       3  0.997977         young
1539      1  0.998263         youth

[348 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 1, 2])

In [128]:
# LDA model
best2_lda_bi = LdaMulticore(corpus = corpus_bi, 
                           num_topics = 6, 
                           id2word = id2word_bi, 
                           random_state = 100,
                           chunksize = 100,
                           passes = 10, 
                          alpha = 'asymmetric',
                          eta = 0.61)

#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(best2_lda_bi, corpus_bi, id2word_bi)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.040196 -0.091825       1        1  37.779776
2      0.241466 -0.021896       2        1  17.324706
5     -0.084002 -0.225701       3        1  14.231345
1     -0.242838  0.073240       4        1  11.482979
3     -0.005450  0.101147       5        1  10.635732
4      0.050628  0.165035       6        1   8.545462, topic_info=              Term          Freq         Total Category  logprob  loglift
160           tree   3310.000000   3310.000000  Default  30.0000  30.0000
2    climatechange  59766.000000  59766.000000  Default  29.0000  29.0000
80         climate  16047.000000  16047.000000  Default  28.0000  28.0000
452          fight   3535.000000   3535.000000  Default  27.0000  27.0000
79          change   8023.000000   8023.000000  Default  26.0000  26.0000
..             ...           ...           ...      ...      ...      ...
371        science    329.718244   2375.090938   Topic6  -5.4173   0.4852
55          planet    310.414321   2461.768359   Topic6  -5.4777   0.3890
500  climateaction    335.784848   5213.441836   Topic6  -5.3991  -0.2828
105           time    317.775827   3529.854198   Topic6  -5.4542   0.0521
216         carbon    279.371363   2599.038021   Topic6  -5.5830   0.2294

[376 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
832       1  0.006061  absorb
832       2  0.006061  absorb
832       3  0.006061  absorb
832       4  0.981813  absorb
832       5  0.006061  absorb
...     ...       ...     ...
907       2  0.002315   young
907       3  0.004629   young
907       4  0.002315   young
907       5  0.025462   young
907       6  0.002315   young

[1703 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 6, 2, 4, 5])